In [22]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from env import user, host, password
from scipy import stats
import csv

np.random.seed(29)





# Have click rates increased post redseign? (Is redsign good?)

Null hypothesis: Click rates have not increased since redesign

Alternative hypothesis: The click rate rate has changed (positive -- good, negative -- bad)

True Positive: Null hypothesis rejected. Click rate has changed (Positive, Negative)

Type I Error: Click rate change has not changed, but we decide that is has. 

True Negative: Null hypothesis not rejected. Click rate has not changed.

Type II Error: Click rate has changed, but we determine that is has not. 


# Is our television ad driving more sales?

Null Hypothesis: Sales have not increased due to TV ad.

Alternative Hypothesis: Sales have changed.

True Positive: Ads have had a positive affect on sales.

True Negative: Sales have not changed (increased) due to TV ad.

Type 1 Error: Sales indicate that the ad has had positive affect when it has not.

Type 2 Error: Results indicate that the ad has not had a positive effect on sales, when it has. 

# Has the network latency gone up since we switched internet service providers?

Null hypothesis: The change of internet service provider has had no effect on latency.

Alternative hypothesis: The change of internet service provider has increased our latency.

True Positive: Change of provider increased latency.

True Negative: No proof that change of provider had the effect of increasing latency.

Type1 Error: Results say that change of ISP increased latency while in reality it did not.

Type2 Error: Results say that change of ISP had no affect on latency while in reality it did.

# Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. 
# A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. 
# A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. 
# Use a .05 level of significance.

In [13]:
sample_1 = stats.norm(90, 15).rvs(40)
sample_2 = stats.norm(100, 20).rvs(50)

stats.ttest_ind(sample_1,sample_2)

# Average time is different, p-value of 0.0015

Ttest_indResult(statistic=-0.9818269584843405, pvalue=0.32887677054120024)

In [14]:
from pydataset import data

data = data('mpg')

df = pd.DataFrame(data)

df2 = df.assign(average_gas = (df.cty + df.hwy)/2)
Gas_1999 = df2[df2.year == 1999].average_gas
Gas_2008 = df2[df2.year == 2008].average_gas

stats.ttest_ind(Gas_1999, Gas_2008)

compact = df2[df2['class'] == 'compact'].average_gas
not_compact = df2[df2['class'] != 'compact'].average_gas
not_compact

compact_test = stats.ttest_ind(compact, not_compact)

In [42]:
x = data.displ
y = data.cty

stats.pearsonr(x,y)

(-0.7985239689348551, 4.737914890205637e-53)

# Use the telco_churn data. Does tenure correlate with monthly charges? Total charges? What happens if you control for phone and internet service?




In [15]:
def get_db_url(username, hostname, password, db_name):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{db_name}'

query = '''
    select tenure, monthly_charges from customers
'''

url = get_db_url(user,host,password,'telco_churn')
charges = pd.read_sql(query,url)
x = charges.tenure
y = charges.monthly_charges

stats.pearsonr(x,y)

(0.24789985628615013, 4.0940449915016345e-99)

# Total charges?


In [16]:


charges = pd.read_excel('telco_churn.xlsx', index_col=0)

charges
charges.dtypes
charges.dtypes
charges.total_charges.fillna(0, inplace = True)
charges.Tenure_2.fillna(0, inplace = True)
charges[charges.total_charges.isnull() == True]

x = charges.Tenure_2

y = charges.total_charges


stats.pearsonr(x, y)


(0.8264581130661657, 0.0)

In [17]:
def get_db_url(username, hostname, password, db_name):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{db_name}'

url = get_db_url(user, host, password, 'employees')
query = '''
select datediff(curdate(),e.hire_date) as tenure,
    salary
    from employees as e
    join salaries as s 
    using(emp_no)
    where s.to_date like "9999%%";
        '''
salary_ten = pd.read_sql(query,url)

/usr/local/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1292, "Incorrect date value: '9999%' for column 'to_date' at row 1")
  result = self._query(query)


In [19]:
salary_ten['tenure'] = salary_ten['tenure'] - salary_ten['tenure'].min()

stats.pearsonr(salary_ten['tenure'],salary_ten['salary']) #Yes, they're correlated


(0.3064625613186075, 0.0)

In [21]:
def get_db_url(username, hostname, password, db_name):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{db_name}'

url = get_db_url(user, host, password, 'employees')



query = '''
select datediff(curdate(),e.hire_date) as tenure,
    count(title) as titles
    from employees as e
    join titles as t 
    using(emp_no)
    group by emp_no;
    '''
titles_by_tenure = pd.read_sql(query,url)
titles_by_tenure['tenure'] = titles_by_tenure['tenure'] - titles_by_tenure['tenure'].min()

stats.pearsonr(titles_by_tenure['tenure'],titles_by_tenure['titles']) # Yes, they're correlated

(0.2665989299136621, 0.0)

In [33]:
# def get_db_url(username, hostname, password, db_name):
#     return f'mysql+pymysql://{username}:{password}@{hostname}/{db_name}'
    
# query = '''SELECT
#     e.*,
#     datediff('2002-09-30', e.hire_date) tenure,
#     t.title,
#     t.from_date title_from,
#     datediff('2002-09-30', t.from_date) t_tenure,
#     et.titles,
#     s.salary,
#     s.from_date salary_from,
#     datediff('2002-09-30', s.from_date) s_tenure,
#     es.salaries
# FROM
#     employees e
# JOIN 
#     titles t 
#     USING(emp_no)
# JOIN 
#     salaries s 
#     USING(emp_no)
# JOIN
#     (SELECT 
#         emp_no,
#         count(*) titles
#     FROM
#         titles
#     GROUP BY
#         emp_no) et
#     USING(emp_no)
# JOIN
#     (SELECT 
#         emp_no,
#         count(*) salaries
#     FROM
#         salaries
#     GROUP BY
#         emp_no) es
#     USING(emp_no)
# WHERE
#     s.to_date > '2002-09-30'
#     AND t.to_date > '2002-09-30';
# '''
# url = get_db_url(user,host,password,'employees')
# employees = pd.read_sql(query,url)

# print(employees.shape)
# print(employees.dtypes)
# employees.sample(5)






In [26]:
from pydataset import data

sleep = data('sleepstudy')
stats.pearsonr(sleep.Reaction,sleep.Days)

(0.5352302262650254, 9.89409632221453e-15)

In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from env import user, host, password
from scipy import stats
import csv

np.random.seed(29)




index = ['Macbook', 'Not Macbook']
columns = ['Codeup', 'Not Codeup']
macbooks = pd.DataFrame([[49,20], [1, 30]], index=index, columns=columns)
stats.chi2_contingency(macbooks)



(36.65264142122487, 1.4116760526193828e-09, 1, array([[34.5, 34.5],
        [15.5, 15.5]]))

In [27]:
# from pydataset import data

# mpg = data('mpg')


# stats.chi2_contingency(pd.crosstab(mpg.trans, mpg.manufacturer))
# # Null: All manufacturers are equally likely to make each transmission type.
# # Alt: All manufacturers are not equally likely to make each transmission type.
# # p-value < 0.05, we reject the null hypothesis

(246.91908570197074,
 7.163203875453598e-10,
 126,
 array([[ 0.38461538,  0.40598291,  0.79059829,  0.53418803,  0.19230769,
          0.2991453 ,  0.17094017,  0.08547009,  0.06410256,  0.08547009,
          0.27777778,  0.10683761,  0.2991453 ,  0.72649573,  0.57692308],
        [ 0.15384615,  0.16239316,  0.31623932,  0.21367521,  0.07692308,
          0.11965812,  0.06837607,  0.03418803,  0.02564103,  0.03418803,
          0.11111111,  0.04273504,  0.11965812,  0.29059829,  0.23076923],
        [ 6.38461538,  6.73931624, 13.12393162,  8.86752137,  3.19230769,
          4.96581197,  2.83760684,  1.41880342,  1.06410256,  1.41880342,
          4.61111111,  1.77350427,  4.96581197, 12.05982906,  9.57692308],
        [ 3.        ,  3.16666667,  6.16666667,  4.16666667,  1.5       ,
          2.33333333,  1.33333333,  0.66666667,  0.5       ,  0.66666667,
          2.16666667,  0.83333333,  2.33333333,  5.66666667,  4.5       ],
        [ 0.46153846,  0.48717949,  0.94871795,  0.641025

In [28]:
query = '''
select gender, dept_name
    from employees
    join dept_emp as de using(emp_no)
    join departments using(dept_no)
    where de.to_date like "9999%%";
    '''

gender_vs_dept = pd.read_sql(query,url)

In [29]:
# dept_filter = gender_vs_dept['dept_name'].apply(lambda x: x in ['Marketing', 'Sales'])
# relevant = gender_vs_dept[dept_filter]
# stats.chi2_contingency(pd.crosstab(relevant.gender, relevant.dept_name)) # No correlation, p-value is .57

(0.3240332004060638,
 0.5691938610810126,
 1,
 array([[ 5893.2426013, 14969.7573987],
        [ 8948.7573987, 22731.2426013]]))

In [32]:
# %matplotlib inline
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from env import user, host, password
# from scipy import stats
# import csv

# np.random.seed(29)




# query = '''
# select gender, count(dm.emp_no) as m_count
#     from employees
#     left join dept_manager as dm using(emp_no)
#     group by emp_no;'''

# gender_vs_manager = pd.read_sql(query,url)

In [31]:
# gender_vs_manager['been_manager'] = gender_vs_manager['m_count'] > 0

# stats.chi2_contingency(pd.crosstab(gender_vs_manager.gender, gender_vs_manager.been_manager)) # No correlation, p-value is .23

NameError: name 'gender_vs_manager' is not defined